In [2]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1)


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
    ],
)

prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock Market Symbol for Cloudflare'}`


Cloudflare might become a trillion-dollar stock by 2050, but a lot could happen over the next 26 years. So instead of fretting over its market cap, investors should pay closer attention to its ... Cloudflare, Inc. - Stock Quote & Chart Stock Quote & Chart Stock Quote NYSE: NET 77.96 Jan 18, 2024 4:00 PM ET Today's Change +0.89 (1.16%) Today's Volume 2,483,613 Today's Open 78.76 Previous Close 77.07 Intraday High 79.68 Intraday Low 76.63 52 Week High 86.91 52 Week Low 39.90 Pricing delayed by 20 minutes. Last Updated Jan 18, 2024 4:00 PM ET Cloudflare, Inc. is a global cloud services provider that delivers a range of services to businesses of all sizes and in all geographies. The Company's network serves as a scalable, unified control plane to deliver security, performance, and reliability across on-premise, hybrid, cloud, and software-as-a-service (S

{'input': "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have.",
 'output': "The stock market symbol for Cloudflare is NET. \n\nCloudflare, Inc. is a global cloud services provider that offers a range of services to businesses of all sizes and in all geographies. Their network serves as a scalable, unified control plane to deliver security, performance, and reliability across various applications.\n\nIn terms of financial performance, Cloudflare's revenue in 2022 was $975.24 million, which is a 48.57% increase compared to the previous year. The company also reduced its losses by 25.71% to -$193.38 million in 2022.\n\nTo analyze if Cloudflare is a potential good investment, it's important to consider various factors such as the company's financial health, growth prospects, competitive landscape, and market trends. Additionally, it's advisable to consult with a financial advisor or conduct furthe